# Runner

This notebook runs the specified notebook on every collection in the specified study.

In order to fetch data from the Cionic REST APIs, you need to be authenticated.  
* Login to [cionic.com](https://cionic.com/a)  
* download a [token file](https://cionic.com/a/v0.7/accounts/@me/token.json)  
* save it to the root of your **cionic-python** directory  

```
note this credential file should never be checked in or shared
```

The default behavior is to run as **prepare_only** 
which will generate the notebooks but not execute them.

Clink on the linked title and run the generated notebook.

## Parameters

```
orgid        | the organization that hosts the study
study        | which study you want to run over    
collection   | optional collection number to run for a specific collection (0 means all)
notebook     | which notebook you want to run                   
execute      | parameterizes the notebook and executes it 
overwrite    | delete previously saved parameterized notebook 
limit        | how many collection records to run
```


In [ ]:
import json
import logging
import os
import re

from IPython.display import clear_output, display
import ipywidgets as widgets
from ipywidgets import Layout, interact

import cionic
from cionic import runner

tokenpath = os.path.abspath('../token.json')
normpath = os.path.abspath('norm_data')
d = cionic.auth(tokenpath=tokenpath)
orgs = [org['shortname'] for org in d]
notebooks = [
    'signal.ipynb',
    'reps.ipynb',
    'gait.ipynb',
    'parse.ipynb',
    'signal-csv.ipynb',
]

org_select = widgets.Dropdown(options=orgs, description="Org ID")
note_select = widgets.Dropdown(options=notebooks, description="Notebook")
study_select = widgets.Dropdown(options=[], description="Study")
protocol_select = widgets.Dropdown(options=[], description="Protocol")
collection_entry = widgets.IntText(description="Collection")
limit_entry = widgets.IntText(value=10, description='Limit')
params_entry = widgets.Textarea(
    rows=5, description="Parameters", layout=Layout(width='800px')
)
run_check = widgets.Checkbox(value=False, description='Execute')
meta_check = widgets.Checkbox(value=False, description='Fetch Meta')
overwrite_check = widgets.Checkbox(value=False, description='Overwrite')

global saved_orgid
saved_orgid = None

global saved_study
saved_study = None

global studies
studies = {}

global protocols
protocols = {}


def _shortname(s):
    return re.sub(r"\s+", "", s['shortname'])


@interact
def show_runner(
    orgid=org_select,
    study=study_select,
    protocol=protocol_select,
    coll=collection_entry,
    notebook=note_select,
    limit=limit_entry,
    extrap=params_entry,
    run=run_check,
    overwrite=overwrite_check,
):
    global saved_orgid, studies, saved_study, protocols

    def on_button_clicked(b):
        with output:
            clear_output()

            gp = {
                'tokenpath': tokenpath,
                'normpath': normpath,
            }

            if extrap:
                try:
                    ep = json.loads(extrap)
                    gp.update(ep)
                except Exception as e:
                    logging.error(e)

            for s in studies:
                if _shortname(s) == study:
                    #
                    # fetch collections
                    #
                    kwargs = {'sxid': s['xid']}
                    if coll != 0:
                        kwargs['num'] = coll
                    for p in protocols:
                        if p['title'] == protocol:
                            kwargs['protoxid'] = p['xid']

                    collections = cionic.get_cionic(f'{orgid}/collections', **kwargs)

                    parameters = {'orgid': orgid, 'study': s['shortname']}
                    parameters.update(gp)

                    if collections:
                        outdir = f'../recordings/{orgid}/{study}'
                        runner.run_collections(
                            notebook,
                            orgid,
                            study,
                            collections,
                            outdir,
                            prepare_only=(not run),
                            overwrite=overwrite,
                            limit=limit,
                            parameters=parameters,
                        )

                    else:
                        print(f"NO COLLECTIONS in {study} for {protocol}")

    run_button = widgets.Button(description='Run')
    run_button.on_click(on_button_clicked)
    output = widgets.Output()
    display(run_button, output)

    if saved_orgid != orgid:
        saved_orgid = orgid
        studies = cionic.get_cionic(f'{orgid}/studies')
        titles = [_shortname(s) for s in studies]
        study_select.options = titles
        saved_orgid = orgid
        return

    print(saved_study, study)

    if saved_study != study:
        saved_study = study
        protocol_titles = [f"__{study}__"]
        for s in studies:
            if _shortname(s) == study:
                protocols = cionic.get_cionic(f"{orgid}/protocols?sxid={s['xid']}")
                for p in protocols:
                    protocol_titles.append(p['title'])
        protocol_select.options = protocol_titles
        return